In [183]:
import requests as rq 
import json
from bs4 import BeautifulSoup as bs
import urllib3


In [3]:
urlhead = "https://www.drugbank.ca/unearth/q?"
frag1 = "utf8=%E2%9C%93&"
frag2 = "searcher=indications&"
query = "query="
filtersFrag= "&us=0&ca=0&eu=0&eu=1&commit=Apply+Filter"

In [198]:
params={'utf8':'%E2%9C%93' , 'searcher' : 'indications' , 'us':'0','ca':'0','eu':'1','commit':'Apply+Filter' , 'query':''}

In [230]:
# disease = "chronic+fatigue+syndrome"
params['query']="\"crohn's+disease\""
# url = urlhead + frag1 + frag2 + query + disease + filtersFrag
response = rq.get(urlhead,params=params)

In [231]:
soup = bs(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

In [233]:
synonym = soup.find_all('div', class_ = 'search-highlights')

In [234]:
drug_matches = soup.find_all('div', class_ = 'db-matches')

In [235]:
len(drug_matches) , len(synonym)

(6, 6)

In [236]:
# synonym

In [237]:
first = drug_matches[3]
first.text

u'Adalimumab'

In [238]:
MatchJson = {'Synonyms':[] , 'MatchedDrugs':[]}
for i in range(len(synonym)):
    syn = synonym[i]
    drug = drug_matches[i]
    synText = syn.text.replace(u'\u2026','')
    drugsName = drug.text.split('/')
    MatchJson['Synonyms'].append(synText)
    MatchJson['MatchedDrugs'].append(drugsName)

In [239]:
MatchJson

{'MatchedDrugs': [[u'Certolizumab pegol ', u' Vedolizumab'],
  [u'Certolizumab pegol ', u' Vedolizumab'],
  [u'Adalimumab ',
   u' Azathioprine ',
   u' Hydrocortisone ',
   u' Infliximab ',
   u' Mercaptopurine ',
   u' Mesalazine ',
   u' Methotrexate ',
   u' Prednisolone ',
   u' Prednisone ',
   u' Sulfasalazine ',
   u' Triamcinolone'],
  [u'Adalimumab'],
  [u'Adalimumab'],
  [u'Betamethasone ',
   u' Bupivacaine ',
   u' Cortisone acetate ',
   u' Dexamethasone ',
   u' Methylprednisolone']],
 'Synonyms': [u"Matched Synonyms:  Severe Crohn's Disease  Matched Title:  Severe Crohn's Disease  ",
  u"Matched Synonyms:  Moderate Crohn's disease  Matched Title:  Moderate Crohn's disease  ",
  u"Matched Synonyms:  CROHN'S DISEASE ... Crohn's Disease(CD) ... Crohn's Disease (CD) ... Crohn's disease NOS  Matched Title:  Crohn's Disease (CD)  ",
  u"Matched Synonyms:  Pediatric Crohn's Disease  Matched Title:  Pediatric Crohn's Disease  ",
  u"Matched Synonyms:  Moderately to Severely Act

In [ ]:
https://www.drugbank.ca/unearth/q?utf8=&searcher=indications&query="kidney+stones"

In [281]:
DiseaseList = [ "bipolar", "\"chronic+fatigue+syndrome\"", "\"crohn's+disease\"" , 'dementia' , 'depression' ,'diabetes' , 'dysautonomia' , 
                'gastroparesis' , 'hypothyroidism' , "\"irritable+bowel+syndrome\"" , "\"interstitial+cystitis\"", "\"kidney+stones\"", "\"Meniere's disease\"" , "\"multiple+sclerosis\"" , 'parkinsons',
                'psoriasis','rheumatoid', "\"sleep+apnea\""]

In [282]:
urlhead = "https://www.drugbank.ca/unearth/q?"
frag1 = "utf8=%E2%9C%93&"
frag2 = "searcher=indications&"
query = "query="
filtersFrag= "&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter"
# params={'utf8':"%E2%9C%93" , 'searcher' : 'indications' , 'us':'0','ca':'0','eu':'0','eu':'1','commit':"Apply+Filter" , 'query':''}

DiseaseDrugJson = {}

for disease in DiseaseList:
    url = urlhead + frag1 + frag2 +query+disease+ filtersFrag
#     params['query']=disease
    response = rq.get(url)
    print response.url
    soup = bs(response.text,'html.parser')
    synonym = soup.find_all('div', class_ = 'search-highlights')
    drug_matches = soup.find_all('div', class_ = 'db-matches')
    print len(drug_matches) , len(synonym)
    MatchJson = {'Synonyms':[] , 'MatchedDrugs':[]}
    try:
        for i in range(len(synonym)):
            syn = synonym[i]
            drug = drug_matches[i]
            synText = syn.text.replace(u'\u2026','')
            drugsName = drug.text.split('/')
            MatchJson['Synonyms'].append(synText)
            MatchJson['MatchedDrugs'].append(drugsName)
    except:
        print "Something went horribly Wrong"
    DiseaseDrugJson[disease] = MatchJson

https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=bipolar&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
11 11
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22chronic+fatigue+syndrome%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22crohn's+disease%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
6 6
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=dementia&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
19 19
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=depression&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
25 25
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=diabetes&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
19 19
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=dysautonomia&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&

In [283]:
for k in DiseaseDrugJson:
    print k
    print len(DiseaseDrugJson[k]['Synonyms'])

psoriasis
25
rheumatoid
13
"multiple+sclerosis"
11
"sleep+apnea"
1
"interstitial+cystitis"
2
"irritable+bowel+syndrome"
5
parkinsons
1
dysautonomia
0
"kidney+stones"
1
"Meniere's disease"
1
bipolar
11
hypothyroidism
1
"crohn's+disease"
6
"chronic+fatigue+syndrome"
0
dementia
19
gastroparesis
2
depression
25
diabetes
19


In [284]:
with open('DiseasesMatched.json','wb') as f:
    json.dump(DiseaseDrugJson,f)

In [285]:
DiseaseDrugJson

{'"Meniere\'s disease"': {'MatchedDrugs': [[u'Betahistine ',
    u' Dimenhydrinate ',
    u' Streptomycin']],
  'Synonyms': [u"Matched Synonyms:  Meniere's Disease ... Meniere's disease, unspecified  "]},
 '"chronic+fatigue+syndrome"': {'MatchedDrugs': [], 'Synonyms': []},
 '"crohn\'s+disease"': {'MatchedDrugs': [[u'Certolizumab pegol ',
    u' Vedolizumab'],
   [u'Certolizumab pegol ', u' Vedolizumab'],
   [u'Adalimumab ',
    u' Azathioprine ',
    u' Hydrocortisone ',
    u' Infliximab ',
    u' Mercaptopurine ',
    u' Mesalazine ',
    u' Methotrexate ',
    u' Prednisolone ',
    u' Prednisone ',
    u' Sulfasalazine ',
    u' Triamcinolone'],
   [u'Adalimumab'],
   [u'Adalimumab'],
   [u'Betamethasone ',
    u' Bupivacaine ',
    u' Cortisone acetate ',
    u' Dexamethasone ',
    u' Methylprednisolone']],
  'Synonyms': [u"Matched Synonyms:  Severe Crohn's Disease  Matched Title:  Severe Crohn's Disease  ",
   u"Matched Synonyms:  Moderate Crohn's disease  Matched Title:  Modera

In [279]:
def cleanDictionary(diseaseList , DiseaseMatch):
    for k in diseaseList:
        for i in range(len(DiseaseMatch[k]['Synonyms'])):
            findString = k.replace('+',' ').lower()
            sourceString = DiseaseMatch[k]['Synonyms'][i].lower()
            if sourceString.find(findString) < 0:
                print "found pointless synonym"
                print DiseaseMatch[k]['Synonyms'][i]

In [280]:
cleanDictionary(DiseaseList,DiseaseDrugJson)

found pointless synonym
Matched Synonyms:  Severe Crohn's Disease  Matched Title:  Severe Crohn's Disease  
found pointless synonym
Matched Synonyms:  Moderate Crohn's disease  Matched Title:  Moderate Crohn's disease  
found pointless synonym
Matched Synonyms:  CROHN'S DISEASE ... Crohn's Disease(CD) ... Crohn's Disease (CD) ... Crohn's disease NOS  Matched Title:  Crohn's Disease (CD)  
found pointless synonym
Matched Synonyms:  Pediatric Crohn's Disease  Matched Title:  Pediatric Crohn's Disease  
found pointless synonym
Matched Synonyms:  Moderately to Severely Active Crohn's Disease  Matched Title:  Moderately to Severely Active Crohn's Disease  
found pointless synonym
Matched Synonyms:  Crohn's disease ... Crohn's disease NOS  
found pointless synonym
Matched Synonyms:  Irritable Bowel Syndrome ... Irritable Bowel Syndrome With Diarrhea ... Diarrhea- Predominant Irritable Bowel Syndrome ... Diarrhea Dominant Irritable Bowel Syndrome ... Diarrhea-predominant Irritable Bowel Syndr